In [1]:
pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.4/35.4 MB 27.7 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np
from keras.models import load_model

model = load_model('my_model.h5')

In [9]:
def preprocess_image(image):
    # Preprocess the image
    resized_image = cv2.resize(image, (28, 28))
    grayscale_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    normalized_image = grayscale_image / 255.0
    reshaped_image = normalized_image.reshape(1, 28, 28, 1)
    return reshaped_image

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # Check if the frame was successfully captured
    if not ret:
        print("Error: Failed to capture frame")
        break
    
    # Preprocess the frame
    processed_frame = preprocess_image(frame)
    
    # Make prediction
    prediction = model.predict(processed_frame)
    predicted_letter = chr(prediction.argmax() + 65) # Assuming output is in range 0-25 corresponding to A-Z
    
    # Display prediction
    cv2.putText(frame, predicted_letter, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Display the frame
    cv2.imshow('ASL Prediction', frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()

Error: Failed to capture frame
